# 0: Loading Dependencies

We load sagemaker because thats what will be doing our computation and we need it to interface with AWS. Scipy, numpy, pandas, and the sklearn cosine similarity function as they will help with our computations and processing. Jikan lets us interface with myanimelist. Boto3 and json are there to help us interface with sagemaker. 

In [ ]:
import sagemaker
import scipy as sp
import numpy as np
import pandas as pd
from jikanpy import Jikan
import boto3, json, io
from sklearn.metrics.pairwise import cosine_similarity

# 1: Preprocessing

First we load our animelist file to a dataframe, where we'll do some preprocessing and get some information about our dataset, namely the number of users and shows. Then we'll also split out data into a test and train set.

I download this data from: https://www.kaggle.com/azathoth42/myanimelist

In [ ]:
animeList = pd.read_csv('animelists_cleaned.csv', usecols = ['username', 'anime_id', 'my_score'])
nAnime = animeList['anime_id'].nunique()
nUsers = animeList['username'].nunique()
animeList = animeList[animeList.my_score != 0]

In [ ]:
print(nAnime, nUsers)
animeList.head()

Next, we want to scale all of our users ratings around their respective average rating. This is just because a user who has an average rating of a 5/10 and another user who averages a 7/10 rating should be treated the same. From here one, we'll use scaledList as our main source of data.

In [ ]:
MeanRating = animeList.groupby('username').mean()
MeanRating['meanRating'] = MeanRating['my_score']
MeanRating.drop(['anime_id', 'my_score'], axis = 1, inplace = True)

stdRating = animeList.groupby('username').std(ddof = 0)
stdRating['stdRating'] = stdRating['my_score']


stdRating.drop(['anime_id', 'my_score'], axis = 1, inplace = True)
stdRating['stdRating'] = stdRating['stdRating'].replace(to_replace = 0.0, value = 1.0)
MSList = pd.merge(stdRating, MeanRating, on = 'username')

scaledList = pd.merge(animeList, MSList, on = 'username')
scaledList['my_score'] = (scaledList['my_score'] - scaledList['meanRating'])/(scaledList['stdRating'])
scaledList.drop(['stdRating', 'meanRating'], axis = 1, inplace = True)

del(MeanRating)
del(stdRating)
del(animeList)

In [ ]:
Now scaledList is the only list we use.

In [ ]:
nColsUsed = len(scaledList.index)

nFeatures = nAnime +nUsers
nTrain = int(nColsUsed*.95)
nTest = 958432

Something to note is that there are significantly more shows than people are watching. The maximum animeId is arund 30,000, but in reality there are only around 7000 shows people have watched. In a sparse format this isn't a big deal. But later we will have to convert it to an array to pass it to AWS. At that point in time, it will be important that we reduce the number of columns we have. So we write a function that returns a dictionary mapping Id's to their respective 'scaled' values.

We need a similar map from usernames to userIds too (as our dataset doesnt give us userIds) in order to load our info into our dataset.

We could do this step when we're loading the data from the dataset into our sparse matrix. In the interest of modularity though, we do it separately.

In [ ]:
def getAnimeDict(animeList):
    rows = animeList.itertuples()
    animeDict = {}
    numAnime = 0
    for row in rows:
        
        if row[2] not in animeDict:
            animeDict[row[2]] = numAnime
            numAnime += 1
    return animeDict

In [ ]:
animeMap = getAnimeDict(scaledList)

In [ ]:
def getUserDict(animeList):
    rows = animeList.itertuples()
    userDict = {}
    numUsers = 0
    for row in rows:
        if row[1] not in userDict:
            userDict[row[1]] = numUsers
            numUsers += 1
    return userDict

In [ ]:
userMap = getUserDict(scaledList)

Now we want to load our test and train data as a sparse matrices then upload them to AWS. Despite the fact that its sparse, its still pretty huge, so this takes a while to run.

In [ ]:
def loadDataset(pdList, lines, columns, offset, userMap, animeMap):
    X = sp.sparse.lil_matrix((lines, columns)).astype('float32')
    Y = []
    numUsers = 0
    UserDict = {}
    F = pdList[offset:].itertuples()
    #print ('starting')
    i = offset
    
    for row in F:
        if i % 1000000 == 0: print(i)
        if i-offset >= lines: break
        
        username = row[1]
        animeId = row[2]
        rating = row[3]
        
        userId = userMap[username]
        adjustedAnimeId = animeMap[animeId]
            
        X[i-offset,int(userId)-1] = 1
        X[i-offset,int(nUsers)+int(adjustedAnimeId)-1] = 1
        if rating >= 0.0:
            Y.append(1)
        else:
            Y.append(0)
        
        i += 1
        
        
    Y=np.array(Y).astype('float32')
    return X,Y

In [ ]:
X_train, Y_train = loadDataset(scaledList, nTrain, nFeatures, 0, userMap, animeMap)

In [ ]:
X_test, Y_test = loadDataset(scaledList, nTest , nFeatures, nTrain, userMap, animeMap)

In [ ]:
print(X_test[0:5])
print(X_train[0:5])

# 2: Machine Learning on Sagemaker

Now we upload stuff to AWS S3

In [ ]:
output_path = 'YOUR OUTPUT PATH'

train_key = 'train.protobuf'
train_prefix = '{}/train'.format(output_path)

test_key = 'test.protobuf'
test_prefix = '{}/test'.format(output_path)

bucket = 'YOUR BUCKET NAME'

outputPath = 's3://{}/{}/output'.format(bucket, output_path)

access_key_id = 'YOUR AWS_ACCESS KEY ID'
secret_access_key = 'YOUR SECRET ACCESS KEY'

regionName = 'YOUR REGION NAME'

containers = {'us-west-2': '174872318107.dkr.ecr.us-west-2.amazonaws.com/factorization-machines:latest',
              'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/factorization-machines:latest',
              'us-east-2': '404615174143.dkr.ecr.us-east-2.amazonaws.com/factorization-machines:latest',
              'eu-west-1': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/factorization-machines:latest'}

client = boto3.client('runtime.sagemaker', region_name=regionName, aws_access_key_id = access_key_id,
                          aws_secret_access_key= secret_access_key)

boto3Session = boto3.Session( aws_access_key_id = access_key_id,
                          aws_secret_access_key= secret_access_key,
                    region_name = regionName)

AWSSession = sagemaker.session.Session(boto_session = sess1)

AWSSession.sagemaker_runtime_client = client

In [ ]:
def writeDatasetToProtobuf(X, Y, bucket, prefix, key, session):
    buf = io.BytesIO()
    sagemaker.amazon.common.write_spmatrix_to_sparse_tensor(buf, X, Y)
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    
    session.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)
    

In [ ]:
train_data = writeDatasetToProtobuf(X_train, Y_train, bucket, train_prefix, train_key, AWSSession)    

In [ ]:
test_data  = writeDatasetToProtobuf(X_test, Y_test, bucket, test_prefix, test_key, AWSSession) 

In [ ]:
fm = sagemaker.estimator.Estimator(containers[AWSSession.boto_session.region_name],
                                   train_instance_count=1,
                                   role = "arn:aws:iam::915797848381:role/service-role/AmazonSageMaker-ExecutionRole-20181008T180358",
                                   train_instance_type='ml.m5.large',
                                   output_path=output_prefix,
                                   sagemaker_session=AWSSession)

fm.set_hyperparameters(feature_dim=nFeatures,
                      predictor_type='binary_classifier',
                      mini_batch_size=1000000,
                      num_factors=64,
                      epochs=25)

fm.fit({'train': train_data, 'test': test_data})

In [ ]:
fm.deploy(instance_type='ml.t2.medium', initial_instance_count=1, endpoint_name = 'AnimeFactorizationMachine')

# 3: Interacting with our deployed model

In [ ]:
fm_predictor = sagemaker.predictor.RealTimePredictor(endpoint = 'AnimeFactorizationMachine', 
                                                     sagemaker_session= AWSSession)

In [ ]:
def fm_serializer(data):
    js = {'instances': []}
    for row in data:
        js['instances'].append({'features': row.tolist()})
    #print js
    return json.dumps(js)

fm_predictor.content_type = 'application/json'
fm_predictor.serializer = fm_serializer
fm_predictor.deserializer = sagemaker.predictor.json_deserializer

Running the following code results in predictions on our test set (or at least 10 of its elements)

In [ ]:
result = fm_predictor.predict(X_test[1000:1010].toarray())
print(result)
print (Y_test[1000:1010])

What if we want reccomendations for a particular user though? We need to do some more work. Right now our factorization machine is able to predict whether or not users that we trained our model on will like certain shows. Because our dataset doesn't contain all the users on myanimelist, we need to find a way for it to make predictions about arbitrary users.

To do this, we essentially use collaborative filtering. We find the user who's most similar to the inputted user, then make predictions for that user. Because our model returns a probability that the user will like a certain show, we just take the shows with the highest probabilities as our recommendation!

In [ ]:
#
# This returns a sparse row vector representing the shows and (normalize) scores a user has watched
# We will use this to find the user most similar to the user we're looking for.
#

def getUserMtrx(username):
    jk = Jikan()
    userList = jk.user(username = username, request = 'animelist')['anime']
    userMtrx = sp.sparse.lil_matrix((1,nAnime))
    L = []
    for show in userList:
        L.append(show['score'])
    m = np.mean(L)
    s = np.std(L)
    for show in userList:
        if show['mal_id'] in animeMap:
            userMtrx[0, animeMap[show['mal_id']]] = (show['score']-m)/s
                
    return sp.sparse.csr_matrix(userMtrx)

In [ ]:
#
# This creates a sparse pivot matrix representing all the users and shows we trained on
# We will use this to find the user most similar to the user we're looking for.
#

def createPivotSparse(pdDF):
    d = {}
    a = 0
    X = sp.sparse.lil_matrix((nUsers,nAnime)).astype('float32')
    userId = 0
    for row in pdDF.itertuples():
        if row[1] not in d:
            d[row[1]] = a
            userId = a
            a = a +1
        else:
            userId = d[row[1]]
        X[userId, animeMap[row[2]]] = row[3]
    return X

In [ ]:
#
# this returns the number of the user who's most similar to the inputted user
# It does this by computing the cosine similarity between the user and the other users
#
def mostSimilarUser(username):
    userList = getUserMtrx(username)
    
    X = createPivotSparse(scaledList)
    
    userSimilarity = cosine_similarity(X, userList, dense_output = False)
    result = userSimilarity.argmax()
    return result

In [ ]:
#
# This returns the sparse output corresponding to the shows the user has not watched
#
def getAnimeList(username, mostSimilarUser):
    jk = Jikan()
    userList = jk.user(username = username, request = 'animelist')['anime']
    showsWatched = {}
    for show in userList:
        if show['mal_id'] in animeMap:
            Aid = animeMap[show['mal_id']]
            showsWatched[Aid] = show['title']
        
    unwatchedTop = []
    for p in range(1,5):
        top_anime = jk.top(type='anime', page=p)['top']
        for show in top_anime:
            if show['mal_id'] in animeMap:
                aId = animeMap[show['mal_id']]
                if aId not in showsWatched:
                    unwatchedTop.append(aId)
                
    X = sp.sparse.lil_matrix((len(unwatchedTop), nFeatures)).astype('float32')
    for x in range(len(unwatchedTop)):
        X[x, 0] = 1
        X[x, unwatchedTop[x]] = 1
        
    return X

In [ ]:
#
# This runs our prediction algorithm on the sparse output corresponding to the shows
# the user has not watched and returns the top 5 results
#
def indicesOfBestAnime(X):
    weights = []
    l = X.shape[0]
    indexOfResult = 0
    for i in range(0,l,5):
        pred = fm_predictor.predict(X[i:i+5].toarray())['predictions']
        for j,x in enumerate(pred):
            if weights == []:
                weights.append((x['score'],i+j))
            if x['score'] > weights[0][0]:
                
                weights.append((x['score'],i+j))
                if len(weights) > 5:
                    weights = sorted(weights, key = (lambda x: x[0]))[1:6]
                else:
                    weights = sorted(weights, key = (lambda x: x[0]))
        del(pred)
    
    
    return list(map(lambda x: x[1], weights))

In [ ]:
#
# given an index of a show, this returns it's name
#
def indexToAnime(X, i):
    jk = Jikan()
    a = X[i].nonzero()
    for val in animeMap.keys():
        if animeMap[val] == a[1][1]:
            showId = val
            break
    
    showinfo = jk.anime(showId)
    showname = showinfo['title']
    return showname

In [ ]:
#
# This packages all the previous functions
#
def animeRecommend(username):
    i = mostSimilarUser(username)
    X = getAnimeList(username, i)
    indices = indicesOfBestAnime(X)
    animeNames = list(map((lambda i: indexToAnime(X,i)), indices))
    
    return animeNames
    

In [ ]:
recommendations = animeRecommend('dakness989')
print(recommendations)